In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns

import torch

%load_ext autoreload
%autoreload 2

In [3]:
df = pd.read_csv('data/intensity.csv')
# df.fillna(df.interpolate(method="spline", order=2), inplace=True)
df.fillna(0, inplace=True)
df

,datetime,D00-001,D00-002,D00-003,D00-004,D00-005,D00-006,D00-007,D00-008,D00-009,...,I58-009,I58-010,I58-011,I58-012,I58-013,I58-014,I63-001,I63-002,I63-003,I63-004
0,2022-01-17 19:00:00,100,108,123,134,138,122,132.0,130,121,...,2.0,5.0,5.0,7.0,0.0,0.0,12.0,9.0,7.0,3.0
1,2022-01-17 19:15:00,118,108,123,117,115,116,128.0,126,148,...,8.0,7.0,8.0,12.0,0.0,0.0,6.0,9.0,9.0,4.0
2,2022-01-17 19:30:00,81,85,131,93,90,88,86.0,82,100,...,2.0,2.0,2.0,2.0,0.0,0.0,6.0,4.0,3.0,3.0
3,2022-01-17 19:45:00,68,70,103,87,96,103,121.0,123,124,...,1.0,2.0,2.0,4.0,0.0,0.0,15.0,10.0,10.0,8.0
4,2022-01-17 20:00:00,80,79,119,147,140,130,125.0,119,121,...,3.0,3.0,3.0,3.0,0.0,0.0,8.0,8.0,8.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3039,2022-02-18 10:45:00,53,44,64,63,51,47,49.0,44,49,...,3.0,5.0,5.0,9.0,6.0,12.0,32.0,4.0,7.0,1.0
3040,2022-02-18 11:00:00,59,57,74,54,50,42,46.0,48,55,...,3.0,4.0,5.0,10.0,8.0,10.0,14.0,0.0,2.0,1.0
3041,2022-02-18 11:15:00,57,51,73,58,59,44,43.0,49,58,...,9.0,5.0,5.0,12.0,9.0,18.0,19.0,6.0,6.0,4.0
3042,2022-02-18 11:30:00,55,55,63,57,48,37,47.0,39,45,...,1.0,1.0,0.0,5.0,13.0,10.0,22.0,5.0,4.0,3.0


In [26]:
# if more than 5% of time series for a station is missed -> drop the station
mta_pivot = mta_pivot.loc[:, mta_pivot.isna().sum() / len(mta_pivot) < 0.05]
mta_pivot

STATION,1 AV,103 ST,104 ST,110 ST,111 ST,116 ST,121 ST,125 ST,14 ST,145 ST,...,UTICA AV,VAN SICLEN AV,WALL ST,WHITLOCK AV,WILSON AV,WINTHROP ST,WOODHAVEN BLVD,WOODLAWN,YORK ST,ZEREGA AV
DATE,,,,,,,,,,,,,,,,,,,,,
2020-12-26,191.0,317.0,20.0,239.0,70.0,84.0,216.0,782.0,414.0,212.0,...,286.0,431.0,53.0,35.0,299.0,224.0,800.0,362.0,7309.0,44.0
2020-12-27,159.0,366.0,19.0,229.0,87.0,65.0,213.0,822.0,346.0,211.0,...,350.0,410.0,46.0,20.0,253.0,188.0,696.0,349.0,447.0,47.0
2020-12-28,310.0,662.0,36.0,399.0,145.0,139.0,407.0,1226.0,773.0,333.0,...,575.0,679.0,218.0,55.0,478.0,542.0,1079.0,663.0,867.0,100.0
2020-12-29,390.0,678.0,54.0,368.0,157.0,157.0,420.0,1187.0,752.0,394.0,...,563.0,699.0,227.0,48.0,461.0,573.0,1430.0,686.0,896.0,123.0
2020-12-30,402.0,680.0,50.0,407.0,143.0,177.0,445.0,1116.0,756.0,411.0,...,577.0,731.0,238.0,52.0,454.0,538.0,1195.0,696.0,894.0,121.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-20,594.0,916.0,463.0,579.0,372.0,156.0,273.0,1741.0,1176.0,624.0,...,709.0,902.0,450.0,62.0,593.0,762.0,2268.0,875.0,1430.0,199.0
2021-12-21,629.0,856.0,513.0,504.0,180.0,176.0,256.0,1699.0,1147.0,595.0,...,664.0,924.0,463.0,75.0,545.0,761.0,1724.0,872.0,1487.0,193.0
2021-12-22,669.0,886.0,467.0,458.0,325.0,157.0,246.0,677.0,1089.0,586.0,...,640.0,884.0,402.0,71.0,566.0,797.0,465.0,769.0,1418.0,203.0


In [27]:
mta_pivot.fillna(mta_pivot.interpolate(method="spline", order=2), inplace=True)
print('NaNs: ', mta_pivot.isna().sum().sum())
mta_pivot.head()

NaNs:  0


STATION,1 AV,103 ST,104 ST,110 ST,111 ST,116 ST,121 ST,125 ST,14 ST,145 ST,...,UTICA AV,VAN SICLEN AV,WALL ST,WHITLOCK AV,WILSON AV,WINTHROP ST,WOODHAVEN BLVD,WOODLAWN,YORK ST,ZEREGA AV
DATE,,,,,,,,,,,,,,,,,,,,,
2020-12-26,191.0,317.0,20.0,239.0,70.0,84.0,216.0,782.0,414.0,212.0,...,286.0,431.0,53.0,35.0,299.0,224.0,800.0,362.0,7309.0,44.0
2020-12-27,159.0,366.0,19.0,229.0,87.0,65.0,213.0,822.0,346.0,211.0,...,350.0,410.0,46.0,20.0,253.0,188.0,696.0,349.0,447.0,47.0
2020-12-28,310.0,662.0,36.0,399.0,145.0,139.0,407.0,1226.0,773.0,333.0,...,575.0,679.0,218.0,55.0,478.0,542.0,1079.0,663.0,867.0,100.0
2020-12-29,390.0,678.0,54.0,368.0,157.0,157.0,420.0,1187.0,752.0,394.0,...,563.0,699.0,227.0,48.0,461.0,573.0,1430.0,686.0,896.0,123.0
2020-12-30,402.0,680.0,50.0,407.0,143.0,177.0,445.0,1116.0,756.0,411.0,...,577.0,731.0,238.0,52.0,454.0,538.0,1195.0,696.0,894.0,121.0


In [28]:
stations = mta_pivot.columns
stations

Index(['1 AV', '103 ST', '104 ST', '110 ST', '111 ST', '116 ST', '121 ST',
       '125 ST', '14 ST', '145 ST',
       ...
       'UTICA AV', 'VAN SICLEN AV', 'WALL ST', 'WHITLOCK AV', 'WILSON AV',
       'WINTHROP ST', 'WOODHAVEN BLVD', 'WOODLAWN', 'YORK ST', 'ZEREGA AV'],
      dtype='object', name='STATION', length=227)

In [29]:
mta_pivot['weekday'] = mta_pivot.index.to_series().dt.weekday
mta_pivot.head(5)

STATION,1 AV,103 ST,104 ST,110 ST,111 ST,116 ST,121 ST,125 ST,14 ST,145 ST,...,VAN SICLEN AV,WALL ST,WHITLOCK AV,WILSON AV,WINTHROP ST,WOODHAVEN BLVD,WOODLAWN,YORK ST,ZEREGA AV,weekday
DATE,,,,,,,,,,,,,,,,,,,,,
2020-12-26,191.0,317.0,20.0,239.0,70.0,84.0,216.0,782.0,414.0,212.0,...,431.0,53.0,35.0,299.0,224.0,800.0,362.0,7309.0,44.0,5
2020-12-27,159.0,366.0,19.0,229.0,87.0,65.0,213.0,822.0,346.0,211.0,...,410.0,46.0,20.0,253.0,188.0,696.0,349.0,447.0,47.0,6
2020-12-28,310.0,662.0,36.0,399.0,145.0,139.0,407.0,1226.0,773.0,333.0,...,679.0,218.0,55.0,478.0,542.0,1079.0,663.0,867.0,100.0,0
2020-12-29,390.0,678.0,54.0,368.0,157.0,157.0,420.0,1187.0,752.0,394.0,...,699.0,227.0,48.0,461.0,573.0,1430.0,686.0,896.0,123.0,1
2020-12-30,402.0,680.0,50.0,407.0,143.0,177.0,445.0,1116.0,756.0,411.0,...,731.0,238.0,52.0,454.0,538.0,1195.0,696.0,894.0,121.0,2


In [30]:
mta_pivot['time_idx'] = np.arange(len(mta_pivot.index))

In [31]:
mta_melt = pd.melt(mta_pivot, id_vars=['time_idx', 'weekday'], value_vars=stations)
mta_melt

,time_idx,weekday,STATION,value
0,0,5,1 AV,191.0
1,1,6,1 AV,159.0
2,2,0,1 AV,310.0
3,3,1,1 AV,390.0
4,4,2,1 AV,402.0
...,...,...,...,...
82623,359,0,ZEREGA AV,199.0
82624,360,1,ZEREGA AV,193.0
82625,361,2,ZEREGA AV,203.0
82626,362,3,ZEREGA AV,185.0


In [32]:
mta_pivot.to_csv('data/mta_ridership_pivot.csv', index=False)
mta_melt.to_csv('data/mta_ridership_melt.csv', index=False)